### Imports

In [5]:
# Import packages
from llama_index import VectorStoreIndex
    from llama_index import SimpleDirectoryReader
import logging
import sys
from llama_index import ServiceContext, LLMPredictor, OpenAIEmbedding, PromptHelper
from llama_index.llms import OpenAI
from llama_index.text_splitter import TokenTextSplitter
from llama_index.node_parser import SimpleNodeParser
from oepul_chat.readers.custom_pdf_reader import CustomPDFReader
from oepul_chat.readers.custom_html_reader import CustomHTMLReader
from oepul_chat.readers.custom_full_pdf_reader import CustomFullPDFReader
from oepul_chat.rag_oepul_string_query_engine import RAGOEPULStringQueryEngine
from llama_index import SimpleDirectoryReader
import random
from llama_index.schema import MetadataMode
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext
from llama_index.llms import LangChainLLM
from llama_index import download_loader
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.prompts import PromptTemplate
# import QueryBundle
from llama_index import QueryBundle

# import NodeWithScore
from llama_index.schema import NodeWithScore

# Retrievers
from llama_index.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
    KeywordTableSimpleRetriever,
)

from llama_index import (
    VectorStoreIndex,
    SimpleKeywordTableIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
)

from typing import List
import pickle




from IPython.core.display import display, HTML




PDFReader = download_loader("PDFReader")


# Import local library
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))

# Autoreload local library
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/var/folders/5l/vkbvbmf94dg7fjtyqjprh7940000gn/T/ipykernel_66528/3642679890.py:50: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### Utilities

In [6]:
def view_response(obj):
    export = """<div style="font-size: 14px;line-height: 1.5;"><strong>Antwort</strong>:<br>"""
    export += f"{obj.response}<br><br>"
    export += """<strong>Quellen:</strong><br>"""
    export += """<ul>"""

    for source_node in obj.source_nodes:
        export += f"<li>"
        export += f"<strong>{source_node.node.metadata['File Name']}</strong><br>"

        export += f"<strong>Header Path</strong>: {source_node.node.metadata['Header Path']}<br>"
        export += f"<strong>Score</strong>: {source_node.score}<br>"
        export += f"<strong>Text</strong>: <i>{source_node.node.text}</i><br>"
        # export += f"<strong>Metadata</strong>: {source_node.node.metadata}<br>"
        export += "<br></li>"

    export += """</ul></div>"""

    display(HTML(export))


def view_docs(docs):
    export = """<div style="font-size: 14px;line-height: 1.5;"><ul>"""
    for doc in docs:
        export += f"<li>"
        export += f"<strong>{doc.metadata['File Name']}</strong><br>"
        export += f"<strong>Header Path</strong>: {doc.metadata['Header Path']}<br>"
        export += f"<strong>Text</strong>: <i>{doc.text}</i><br>"
        # export += f"<strong>Metadata</strong>: {source_node.node.metadata}<br>"
        export += "<br></li>"
    export += """</ul></div>"""
    display(HTML(export))


def load_data(filepath, filetype, reader):
    """Load markdown docs from a directory, excluding all other file types."""
    print(f'loading data... {filepath}')
    loader = SimpleDirectoryReader(
        input_dir=filepath,
        file_extractor={filetype: reader},
        recursive=True
    )

    data = loader.load_data()

    # print short summary
    print("Loaded {} documents".format(len(data)))
    print("First document metadata: {}".format(data[1].metadata))
    print("First document text: {}".format(data[1].text[0:80]))

    return data

### Logging setup

In [9]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Data


### Data preprocessing, structure extraction
I built a CustomPDFReader which can be found in `oepul_chat/custom_pdf_reader.py` it extracts the structure out of the PDF and embeds it in the metadata field `header_path` i think this is one od the first crucial steps as it gives each text element a context in all of the files. With llama index we can then give this fiel to the retriever or the LLM or both. 

In [12]:
# load all official OEPUL docs with custom PDF reader
oepul_official_docs = load_data("data/OEPUL_PDF/", ".pdf", CustomPDFReader())
# # load html guide from BIO Austria
bio_austria_guide = load_data("data/BIO_AUSTRIA", ".html", CustomHTMLReader())
# # Load rest of AMA docs with simple pdf reader
ama_official_docs = load_data("data/AMA", ".pdf", PDFReader())

# merge documents lists
docs_list = [oepul_official_docs]#[oepul_official_docs, ama_official_docs, bio_austria_guide]
documents = [doc for docs in docs_list
             for doc in docs]

# write docs to pickle
with open('data/documents.pickle', 'wb') as f:
    pickle.dump(documents, f)

# import docs pickle
with open('data/documents.pickle', 'rb') as f:
    documents = pickle.load(f)

loading data... data/OEPUL_PDF/
Loaded 442 documents
First document metadata: {'File Name': 'O6_14_Almbewirtschaftung_2023_04.pdf', 'Content Type': 'text', 'Header Path': 'Almbewirtschaftung/ÖPUL 2023'}
First document text: Almbewirtschaftung STAND April 2023
loading data... data/BIO_AUSTRIA
Loaded 38 documents
First document metadata: {'File Name': 'aktueller-planungsstand-zu-bio-im-oepul-2023.html', 'Content Type': 'text', 'Header Path': '', 'tag': 'p'}
First document text: Am 13.09.2022 wurde der österreichische GAP – Strategieplan, in dem die Ausgesta
loading data... data/AMA
Loaded 148 documents
First document metadata: {'page_label': '2', 'file_name': '20230131_Merkblatt_MFA2023_V3.pdf'}
First document text: Merkblatt Mehrfachantrag 2023  Seite 2 von 53 www.eama.at  | www.ama.at   
 EDIT


In [16]:
test_docs = oepul_official_docs = load_data(
    "data/OEPUL_PDF/", ".pdf", CustomFullPDFReader())

loading data... data/OEPUL_PDF/
Loaded 15 documents
First document metadata: {'File Name': 'O6_15_Tierwohl-Behirtung_2023_04.pdf', 'Content Type': 'text'}
First document text: ## ÜBERSICHT
Die Prämie wird für die Behirtung von Raufutterverzehrern auf Almwe


[Document(id_='5af2c7ed-a9ec-4882-a262-0833cfc63f29', embedding=None, metadata={'File Name': 'O6_14_Almbewirtschaftung_2023_04.pdf', 'Content Type': 'text'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='1fec3828f0763f1763f51e22e20c3ffa3c8947cc9cab21c95629ec4d7867f95a', text='## ÜBERSICHT\nDie Prämie wird für Almweideflächen gewährt, die mit Tieren bestoßen werden. Gefördert werden Kosten und Einkommensverluste, die durch die Einhaltung der Verpflichtungen, insbesondere durch den höheren Arbeitszeitbedarf für Weidepflege und den Verzicht auf Mineraldünger sowie chemischen Pflanzenschutz entstehen. Optional erfolgt ein Prämienzuschlag für naturschutzfachlich begründete Auflagen. Die Maßnahme ist von der Almbewirtschafterin oder dem Almbewirtschafter zu beantragen.\n\n## ZIELSETZUNG\nDie Maßnahme dient dem Erhalt der Kulturlandschaft und dem Schutz der Biodiversität durch standortangepasste Land- und Forstwirtschaft. Durch Forcierung der Berücksi

In [29]:
test_docs_sub = test_docs[0:1]

In [30]:
from llama_index import DocumentSummaryIndex, get_response_synthesizer

chatgpt = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=chatgpt, chunk_size=1024)

summary_prompt = PromptTemplate(
    "Du bist ein System welches Zusammenfassungen von Maßnahmen für Landwirte in Österreich aus dem Programm Österreichischen Programm für umweltgerechte Landwirtschaft kurz OEPUL erstellt.\n"
    "Hier die Informationen zu den ÖPUL Förderungen/ Maßnahmen:\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Fasse die Maßnahme zusammen, achte besonders auf die Bedingungen und Förderhöhen.\n"
    "Der Landwirt sollte schnell erfassen können, ob die Maßnahme für ihn in Frage kommt.\n"
    "Zusammenfassung: "
)



# default mode of building the index
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    summary_template=summary_prompt
)

doc_summary_index = DocumentSummaryIndex.from_documents(
    test_docs_sub,
    service_context=service_context,
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

Summarizing documents:   0%|          | 0/1 [00:00<?, ?it/s]

current doc id: 5af2c7ed-a9ec-4882-a262-0833cfc63f29
INFO:llama_index.indices.document_summary.base:> Generated summary for doc 5af2c7ed-a9ec-4882-a262-0833cfc63f29: Die Maßnahme "Almbewirtschaftung" im Rahmen des Österreichischen Programms für umweltgerechte Landwirtschaft (ÖPUL) fördert den Erhalt der Kulturlandschaft und den Schutz der Biodiversität. Um an der Maßnahme teilzunehmen, müssen Landwirte mindestens 3,00 ha Almweideflächen mit mindestens 3,00 raufutterverzehrenden Großvieheinheiten (RGVE) bewirtschaften. Die Mindestvertragslaufzeit beträgt 4 Jahre, optional mit Zuschlag bis zum 31. Dezember 2028. 

Die Tiere müssen mindestens 14 Tage vor dem Meldedatum auf der Alm sein und sich dort Tag und Nacht aufhalten. Der maximale Viehbesatz beträgt 2,00 RGVE pro Hektar Almweidefläche und es werden nur Tiere angerechnet, die mindestens 60 Tage auf der Alm sind. Eine Zufütterung mit Grundfutter ist grundsätzlich nicht erlaubt, jedoch ist eine Ausgleichsfütterung mit Heu, Mineralstoff

Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.52it/s]


In [34]:
doc_summary_index.get_document_summary(test_docs_sub[0].id_)

320

In [44]:
# Show some example
print(random.sample(oepul_official_docs, 1)[0].get_content(
    metadata_mode=MetadataMode.ALL))

File Name: O6_20_Tierwohl-Weide_2023_04.pdf
Content Type: text
Header Path: Tierwohl – Weide/FÖRDERBEDINGUNGEN/ZUGANGSMÖGLICHKEIT DER TIERE ZU TRÄNKE UND UNTERSTELL-/MÖGLICHKEIT

Für die Dauer der Weidehaltung muss für die Weidetiere eine Zugangsmöglichkeit zu einer Tränke und eine Unterstellmöglichkeit (oder Möglichkeit der raschen Verbringung in den Stall, wenn notwendig) bestehen. Als Unterstellmöglichkeit kann beispielsweise auch eine Baumgruppe dienen.


Llama index support different content outputs per Document or Node depending on if the retriever or the LLM is accessing the document. For now we will exlude the file name as its mostly duplicate information.

In [65]:
# Hide the File Name from the LLM
for doc in documents:
    doc.excluded_llm_metadata_keys = ["File Name", "Content Type"]
    doc.excluded_embed_metadata_keys = ["File Name", "Content Type"]



print("# Embed example")
print(random.sample(documents, 100)[0].get_content(
    metadata_mode=MetadataMode.EMBED))

print("\n# LLM example")
print(random.sample(documents, 100)[0].get_content(
    metadata_mode=MetadataMode.LLM))

# Embed example
Header Path: Heuwirtschaft/TEILNAHMEVORAUSSETZUNGEN/VERTRAGSZEITRAUM

Der Verpflichtungs- und Vertragszeitraum der Maßnahme beträgt mindestens 4 Jahre und läuft bis 31. Dezember 2028. Beginn Vertragszeitraum 01.01.2023 6 Jahre (bis einschließlich 31.12.2028) 01.01.2024 5 Jahre (bis einschließlich 31.12.2028) 01.01.2025 4 Jahre (bis einschließlich 31.12.2028) Für die zusätzlich beantragbare Option „Verzicht auf Mähaufbereiter“ läuft der Verpflichtungs- und Vertragszeitraum über ein Kalenderjahr (1. Jänner bis 31. Dezember). Informationsblatt ÖPUL 2023 Heuwirtschaft Seite 1 von 5 www.eama.at | www.ama.at 3.2 MAßNAHMENKOMBINATION Es muss zeitgleich entweder an der Maßnahme „Umweltgerechte und biodiversitäts- fördernde Bewirtschaftung” oder „Biologische Wirtschaftsweise” bzw. „Biologische Wirtschaftsweise – Teilbetrieb” teilgenommen werden (Kombinationsverpflichtung).

# LLM example
Header Path: Umweltgerechte und biodiversitätsfördernde Bewirtschaftung/ALLGEMEINE FÖRDERBED

### Further splitting into Chunks
So far the documents have been splitted logically in hierarchies but we want them splitted in chunks this will be done with the Llama index SimpleNodeParser Object. The chunk_size and chunk overlap can be seen as tuning parameters.

In [131]:
node_parser = SimpleNodeParser.from_defaults(
    text_splitter=TokenTextSplitter(chunk_size=700, chunk_overlap=20)
)

### Defining the embedding model

For testing i will use free Embeddings for document retrieval which will later be switched to OpenAI Embeddings.

In [132]:
# embed_model = OpenAIEmbedding()
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base"))

## LLM Pipeline 

Similar to the embeddings for testing a local LLM will be used.

### LLM definition

In [133]:
llm = OpenAI(model='text-davinci-003', temperature=0, max_tokens=512)
# from langchain.llms import LlamaCpp
# from langchain.callbacks.manager import CallbackManager
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# # Callbacks support token-wise streaming
# callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# llm = LangChainLLM(llm=LlamaCpp(
#     model_path="/Users/vali/repos/models/openorca-platypus2-13b.ggmlv3.q4_0.bin",
#     temperature=0.75,
#     max_tokens=2000,
#     top_p=1,
#     callback_manager=callback_manager,
#     verbose=True,  # Verbose is required to pass to the callback manager
# ))

# from llama_index.llms import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

# llm = LlamaCPP(
#     # You can pass in the URL to a GGML model to download it automatically
#     model_url=model_url,
#     # optionally, you can set the path to a pre-downloaded model instead of model_url
#     model_path=None,
#     temperature=0.1,
#     max_new_tokens=256,
#     # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
#     context_window=3900,
#     # kwargs to pass to __call__()
#     generate_kwargs={},
#     # kwargs to pass to __init__()
#     # set to at least 1 to use GPU
#     model_kwargs={"n_gpu_layers": 1},
#     verbose=True,
# )


### Prompt and Pipeline

In [134]:
prompt_helper = PromptHelper(
    context_window=4096,
    num_output=256,
    chunk_overlap_ratio=0.1,
    chunk_size_limit=None
)

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    node_parser=node_parser,
    prompt_helper=prompt_helper,
)

In [135]:
nodes = node_parser.get_nodes_from_documents(documents)

In [136]:
qa_prompt = PromptTemplate(
    "Du bist ein Supportsystem für Landwirte in Österreich und bekommst Informationen zum Österreichischen Programm für umweltgerechte Landwirtschaft kurz OEPUL.\n"
    "Anhand dieser Informationen sollst du Landirten helfen Entscheidungen zu treffen so dass sie Förderungen aus dem ÖPUL Programm bekommen.\n"
    "Hier die Informationen zu den ÖPUL Förderungen:\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Angesichts der Kontextinformationen und ohne Vorwissen beantworte die Frage:\n"
    "Frage: {query_str}\n"
    "Antwort: "
)

# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

# define custom retriever
index = VectorStoreIndex(nodes, storage_context=storage_context, service_context=service_context)

query_engine = index.as_query_engine(
    summary_template=qa_prompt, similarity_top_k=8)


In [141]:
query_engine = index.as_chat_engine(
    summary_template=qa_prompt, similarity_top_k=8)

In [ ]:
resp = query_engine.query("Gibt es Pramien die den Tieren zu gute kommen?")
view_response(resp)

In [142]:
chat_engine = index.as_chat_engine(summary_template=qa_prompt, similarity_top_k=6)
resp = chat_engine.chat("Gibt es Pramien die den Tieren zu gute kommen?")

In [143]:
view_response(resp)

## Try out the pipeline

In [38]:

query_engine = index.as_query_engine(service_context=service_context)

query1 = "Welche Begrünungskulturen sind gültig?"
query2 = "Welche Maßnahme trägt zur Verringerung der Treibhausgasemission bei?"
query3 = "Gibt es Pramien die den Tieren zu gute kommen?"
query4 = "Ich habe letztes Jahr Weizen angebaut"

response = query_engine.query(query3)

view_response(response)